In [9]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sn
import random
plt.close('all')

In [21]:
queries = pd.read_csv('new_logs.csv', index_col="idx", dtype={"AnonID": "Int64", "Query": "string", "QueryTime": "string", "ItemRank": "Int32", "ClickURL": "string", "Type": "string", "SessionNum": "Int32"})


In [22]:
group_by_sessions = queries.groupby('AnonID', sort=False, as_index=False)
qqq = group_by_sessions.max(numeric_only=True)
newIDs = qqq.loc[qqq['SessionNum'] >= 19]
print(newIDs)
# print(newIDs["AnonID"])

         AnonID  SessionNum  ItemRank
0         91602         286       189
1       2846780         192        40
2       1750999         363       364
3        118401         322       119
4        186465         159       182
...         ...         ...       ...
53028  18287420          30      <NA>
53643   3962900          23        35
54960   1466736          20         1
63748   7213879          26         5
64443  16419908          25         1

[15665 rows x 3 columns]


In [12]:
def compare_queries(queryA, queryB):
  # compute semantic similarities and edit distances to gauge query similarities
  return 1

def flatten_logs(logs_dataframe):
  previous_row = None
  previous_query = ''
  flattened = []
  
  for idx, row in logs_dataframe.iterrows():
    if previous_row is None: # Every logstream starts with a query
      print(row)
      flattened.append({
        "type": "NewQuery",
        "query": row['Query']
      })
      previous_row = row
      previous_query = row['Query']
      
      continue

    if row["Type"] == "Query":
      if compare_queries(previous_query, row['Query']) >= 1:
        flattened.append({
          "type": "NewQuery",
          "query": row['Query']
        })
      else:
        flattened.append({
          "type": "RefinedQuery",
          "query": row['Query']
        })
    elif row["Type"] == "Click":
      if row["ItemRank"] == 1:
        flattened.append({
          "type": "Click1",
          "rank": row['ItemRank']
        })
      elif row["ItemRank"] >= 2 and row["ItemRank"] < 6:
        flattened.append({
          "type": "Click2-5",
          "rank": row['ItemRank']
        })
      elif row["ItemRank"] >= 6 and row["ItemRank"] < 10:
        flattened.append({
          "type": "Click6-10",
          "rank": row['ItemRank']
        })
      else:
        flattened.append({
          "type": "Click11+",
          "rank": row['ItemRank']
        })
    elif row["Type"] == "NextPage":
      flattened.append({
        "type": "NextPage"
      })

  return flattened



In [23]:
group_by_sessions = queries.groupby(["AnonID", "SessionNum"])

tuples = []
# for idx, row in newIDs.iterrows():
#   tuples += [(row["AnonID"], i) for i in range(row["SessionNum"])]

## First, extract all tuples with appropriately long sessions

ssss = group_by_sessions.count()
ss = ssss[ssss["Query"] >= 5]

for idx, _ in ss.iterrows():
  tuples += [idx]

### Then, draw 5,000 samples

SAMPLE_SIZE = 5000

sample = random.sample(tuples, SAMPLE_SIZE)


In [36]:
# Markov Model

class MarkovModel(object):
  states = np.ndarray(0)
  num_states = 0
  probabilities = None
  initial_probabilities = np.ndarray(0)

  def __init__(self, states, probabilities = None, initial_probabilities = None):
    self.states = states
    self.probabilities = probabilities
    self.initial_probabilities = initial_probabilities

    if self.probabilities is None:
      self.probabilities = np.ones((len(states), len(states))) / (len(states) ** 2)

  def update_probabilities(self, probabilities):
    self.probabilities = probabilities

  def compute_probability(self, sequence):
    prob = 1
    prev_state_idx = -1
    for idx, state in enumerate(sequence):
      cur_state_idx = self.states.index(state['type'])
      if idx == 0: # initial prob
        prob = prob * self.initial_probabilities[cur_state_idx]
        prev_state_idx = cur_state_idx
      else:
        prob = prob * self.probabilities[prev_state_idx, cur_state_idx]
        prev_state_idx = cur_state_idx

    
    return prob





In [29]:
# https://lovit.github.io/visualization/2019/12/02/som_part1/

def initialize_simple(n_rows, n_cols):

  grid, pairs = make_grid_and_neighbors(n_rows, n_cols)

  x_ranges = np.linspace(0, 1, n_rows)
  y_ranges = np.linspace(0, 1, n_cols)

  C = np.asarray([[x, y] for x in x_ranges for y in y_ranges])

  return grid, C, pairs

def initialize_markov(n_rows, n_cols, states, initial_probabilities):

  grid, pairs = make_grid_and_neighbors(n_rows, n_cols)

  x_ranges = np.linspace(0, 10, n_rows)
  y_ranges = np.linspace(0, 10, n_cols)

  C = np.asarray([MarkovModel(states, None, initial_probabilities) for x in x_ranges for y in y_ranges])


  return grid, C, pairs


def make_grid_and_neighbors(n_rows, n_cols):
  grid = np.arange(n_rows * n_cols).reshape(n_rows, n_cols)
  pairs = []
  for i in range(n_rows):
    for j in range(n_cols):
      idx = grid[i, j]
      neighbors = []
      if j > 0:
        neighbors.append(grid[i, j-1])
      if i > 0:
        neighbors.append(grid[i-1, j])
      for nidx in neighbors:
        pairs.append((idx, nidx))

  return grid, pairs

def make_masks(grid, sigma = 1.0, max_width = 2):
  rows, cols = np.where(grid >= 0)
  data = grid[rows, cols]

  sorted_indices = data.argsort()
  indices = zip(rows[sorted_indices], cols[sorted_indices])
  masks = [make_gaussian_mask(grid, i, j, sigma, max_width) for i, j in indices]
  masks = [mask.flatten() for mask in masks]

  return masks

def make_gaussian_mask(grid, i, j, sigma = 1.0, max_width = 2):
  mask = np.zeros(grid.shape)
  for i_, j_ in zip(*np.where(grid >= 0)):
    if (max_width > 0) and (abs(i - i_) + abs(j - j_) > max_width):
      continue
    mask[i_, j_] = np.exp(-((i-i_)**2 + (j-j_) ** 2) / sigma ** 2) / (2 * np.pi * sigma)

  return mask

def make_neighbor_graph(grid, max_width = 2, decay = 0.25):
  def weight_array(f, s):
    return np.asarray([np.power(f, i) for i in range(1, s+1) for _ in range(4*i)])

  def pertubate(s):
    def unique(i, s):
      if abs(i) == s:
        return [0]
      return [s - abs(i), -s + abs(i)]

    def pertubate_(s_):
      return [(i, j) for i in range(-s, s+1) for j in unique(i, s_)]

    return [pair for s_ in range(1, s+1) for pair in pertubate_(s_)]

  def is_outbound(i_, j_):
    return (i_ < 0) or (i_ >= n_rows) or (j_ < 0) or (j_ >= n_cols)

  n_rows, n_cols = grid.shape
  n_codes = n_rows * n_cols

  W = weight_array(decay, max_width)
  N = -np.ones((n_codes, W.shape[0]), dtype = np.int)
  N_inv = -np.ones((n_codes, W.shape[0]), dtype = np.int)

  for row, (i, j) in enumerate(zip(*np.where(grid >= 0))):
    idx_b = grid[i, j]
    for col, (ip, jp) in enumerate(pertubate(max_width)):
      if is_outbound(i+ip, j+jp):
        continue
      idx_n = grid[i+ip, j+jp]
      N[idx_b, col] = idx_n
      N_inv[idx_n, col] = idx_b
  
  return N, N_inv, W


In [14]:
from sklearn.metrics import pairwise_distances_argmin_min
def closest(X, C, metric):
  # return (idx, dist)

  return pairwise_distances_argmin_min(X, C, metric = metric)

def update_stochastic(X, C, lr = 0.01, metric = 'euclidean', masks = None):
  n_data = X.shape[0]
  n_codes, n_features = C.shape
  C_new = C.copy()

  Xr = X[np.random.permutation(n_data)]

  for i, Xi in enumerate(Xr):
    bmu, _ = closest(Xi.reshape(1, -1), C_new, metric)
    bmu = int(bmu)

    diff = Xi - C_new
    grad = lr * diff * masks[bmu][:, np.newaxis]
    C_new += grad
  
  return C_new
    

def update_cmeans(X, C, update_ratio, metric='euclidean', batch_size = -1, grid = None, neighbors = None, inv_neighbors = None, weights = None, adjust_ratio = 0.5, max_width = 2, decay = 0.25, **kargs):
  if (neighbors in None) or (weights is None):
    neighbors, inv_neighbors, weights = make_neighbor_graph(grid, max_width, decay)

  C_new = C.copy()

  for b, Xb in enumerate(to_minibatch(X, batch_size)):
    C_new = update_cmeans_batch(Xb, C_new, update_ratio, metric, neighbors, inv_neighbors, weights, adjust_ratio)

  return C_new

def update_cmeans_batch(X, C, update_ratio, metric, neighbors, inv_neighbors, weights, adjust_ratio):
  n_data = X.shape[0]
  n_codes = C.shape[0]

  C_cont = np.zeros(shape = C.shape)
  W_new = np.zeros(n_codes)

  bmu, dist = closest(X, C, metric)

  for bmu_c in range(n_codes):
    indices = np.where(bmu == bmu_c)[0]
    n_matched = indices.shape[0]

    if n_matched == 0:
      continue
      
    Xc = np.asarray(X[indices, :].sum(axis=0)).reshape(-1)
    C_cont[bmu_c] += Xc
    W_new[bmu_c] += n_matched

    if weights.shape[0] == 0:
      continue

    for c, w in zip(neighbors[bmu_c], weights):
      if c == -1:
        continue
      C_cont[c] += w * Xc
      W_new[c] += w * n_matched

  C_new = update_ratio * C_cont + (1 - update_ratio) * C
  return C_new


In [37]:
# Markov model specific implementations here

def find_pair_from_sequence(seq, state_prev, state_next):
  for i in range(len(seq) - 1):
    if seq[i]['type'] == state_prev and seq[i+1]['type'] == state_next:
      return 1
    
  return 0

def update_markov(sequences, C, states, grid, min_neighborhood_size = 0, decreasing_factor = 0.95, epochs = 100):

  K = C.size # grid size
  C_flat = C.ravel()
  N = len(sequences) # number of inputs
  m = len(states)


  masks = make_masks(grid, sigma = 1, max_width = 2) # h-matrix, with row vectors as mask for each grid point
  delta_matrix = np.asarray([[1 if sequences[_n][1]['type'] == states[_m] else 0 for _m in range(m)] for _n in range(N)]) # m X N deltas
  beta_matrix = np.asarray([[[find_pair_from_sequence(sequences[_n], states[_prev], states[_next]) for _next in range(m)] for _prev in range(m)] for _n in range(N)]) # m X m X N matrix of betas
  p_c = np.ones((N, K))

  cur_neighborhood_size = 5
  # make_neighbor_graph(grid, max_width =  )

  # E-step: compute Q function and update probabilities

  for i in range(epochs):

    masks = make_masks(grid, sigma = cur_neighborhood_size, max_width = 2) # h-matrix, with row vectors as mask for each grid point
    delta_matrix = np.asarray([[1 if sequences[_n][1]['type'] == states[_m] else 0 for _m in range(m)] for _n in range(N)]) # m X N deltas
    beta_matrix = np.asarray([[[find_pair_from_sequence(sequences[_n], states[_prev], states[_next]) for _next in range(m)] for _prev in range(m)] for _n in range(N)]) # m X m X N matrix of betas

    p = np.asarray([[C[k].compute_probability(sequences[n]) for k in range(K)] for n in range(N)]) # N by K matrix with probs
    

    for n in range(N):
      for k in range(K):
        i, j = np.where(grid == n)
        mask = masks[k]

        for _k in np.where(mask > 0)[0]:
          r = grid[i, j]
          p_c[n, k] = p_c[n, k] * (p[n, r] ** mask[_k])


    # M-step: update params

    # initial states

    for k in range(K):
      denom_matrix = delta_matrix @ p_c[:, k] @ masks[k] # ending up with (m X N) (N X 1) (1 X K) == m X K 
      denom_vector = np.sum(denom_matrix, axis = 1)
      new_initials = denom_vector / np.sum(denom_vector)
      C_flat[k].initial_probabilities = new_initials

    # Transition states

    for k in range(K):
      denom_ndarray = beta_matrix @ p_c[:, k] @ masks[k] # ending up with (m, m, N) (N, 1) (1, K) == m, m, K
      denom_mat = np.sum(denom_ndarray, axis = 2)
      new_transitions = denom_mat / np.sum(denom_mat, axis = 1)[:, None]

      C_flat[k].probabilities = new_transitions

    # Compute and print likelihood

    likelihood = 0

    for n in range(N):
      _sum = np.sum(p_c[n, :]) / K
      likelihood += np.log(_sum)

    if cur_neighborhood_size < min_neighborhood_size:
      break
    
    cur_neighborhood_size = cur_neighborhood_size * decreasing_factor

    print("Current Likelihood: %d" % likelihood)
  

  


In [38]:
sequences = []

for s in sample:
  g = group_by_sessions.get_group(s)
  sequences.append(flatten_logs(g))

states = ["NewQuery", "RefinedQuery", "Click1", "Click2-5", "Click6-10", "Click11+", "NextPage"]

grid, C, pairs = initialize_markov(10, 10, states, [1] + [0 for i in range(len(states) - 1)])


AnonID                     588533
Query         online stock buying
QueryTime     2006-04-18 01:16:03
Type                        Query
SessionNum                     18
ItemRank                     <NA>
ClickURL                     <NA>
Name: 3048148, dtype: object
AnonID                   24661658
Query                 grass seeds
QueryTime     2006-05-29 15:11:26
Type                        Query
SessionNum                      1
ItemRank                     <NA>
ClickURL                     <NA>
Name: 4389377, dtype: object
AnonID                      24418370
Query         ncstateunclaimcash.com
QueryTime        2006-05-25 00:56:05
Type                           Query
SessionNum                        11
ItemRank                        <NA>
ClickURL                        <NA>
Name: 2424390, dtype: object
AnonID                     949253
Query          yorki poo pictures
QueryTime     2006-04-26 23:53:06
Type                        Query
SessionNum                    138
ItemRank

In [39]:

update_markov(sequences, C, states, grid, min_neighborhood_size = 0.5, epochs = 10)

TypeError: where() got an unexpected keyword argument 'grid'